In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# I have done this task with 3 expriments : 
1. Using Keras (Document provided in the README file)
2. Using Word2Vec library (I had previously known about this library so I used it)
3. Using BERT 


# **Experiment 1:** Creating Word Embeddings using Keras# 

In [2]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

2025-10-20 19:27:24.021757: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760988444.285438      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760988444.360124      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [15]:

from datasets import load_dataset
import tensorflow as tf

ds = load_dataset("lucadiliello/newsqa", split="train[:1%]")
texts = [ex["context"] for ex in ds if ex.get("context") and ex["context"].strip()]

text_ds = tf.data.Dataset.from_tensor_slices(texts).batch(256)


max_tokens = 1000
seq_len = 128
vectorize = tf.keras.layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=seq_len,  
)
vectorize.adapt(text_ds)


inp = tf.keras.Input(shape=(1,), dtype=tf.string)
x = vectorize(inp)                           
x = tf.keras.layers.Embedding(1000, 5)(x)    

x = tf.keras.layers.GlobalAveragePooling1D()(x)
out = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inp, out)
model.summary()


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_3            │ (None, 128)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_3 (Embedding)         │ (None, 128, 5)         │         5,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_6      │ (None, 5)              │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,006 (19.55 KB)

 Trainable params: 5,006 (19.55 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:

emb_layer = next(l for l in model.layers if isinstance(l, tf.keras.layers.Embedding))
W = emb_layer.get_weights()[0]             


vocab = vectorize.get_vocabulary()         
words = (["[PAD]"] + vocab)[:W.shape[0]]   


import pandas as pd, os

df = pd.DataFrame({
    "word": words,
    "embedding": [W[i].tolist() for i in range(W.shape[0])]
})

print(df.head())  


    word                                          embedding
0  [PAD]  [0.014685098081827164, -0.03371154144406319, 0...
1         [-0.0024613365530967712, 0.04334351047873497, ...
2  [UNK]  [0.0341651476919651, 0.038995180279016495, 0.0...
3    the  [-0.02043747901916504, 0.029480163007974625, 0...
4     to  [-0.014732073992490768, -0.03665952757000923, ...


# **Experiment 2:** Creating Word Embeddings using Word2Vec# 

In [1]:
!pip install gensim datasets transformers tensorflow scikit-learn matplotlib pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 40.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 41.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following

In [2]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import torch
from transformers import BertTokenizer, BertModel
import tensorflow as tf

2025-10-20 19:34:25.813436: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760988866.056767      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760988866.131015      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
from datasets import load_dataset

# NewsQA dataset
dataset = load_dataset("lucadiliello/newsqa", split="train[:1%]")
texts = [ex["context"] for ex in dataset if ex.get("context") and ex["context"].strip()]
sentences = [t.lower().split() for t in texts]


In [7]:
len(sentences)

742

In [8]:

model_w2v = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4)
words_w2v = list(model_w2v.wv.index_to_key)
embeddings_w2v = [model_w2v.wv[w].tolist() for w in words_w2v]
df_w2v = pd.DataFrame({'word': words_w2v, 'embedding': embeddings_w2v})

df_w2v.to_csv('word2vec_embeddings.csv', index=False)

In [9]:
word2vec_df = pd.read_csv('word2vec_embeddings.csv')
word2vec_df.shape
word2vec_df.head()
     

,word,embedding
0,the,"[-0.572348952293396, 0.4237285256385803, -0.23..."
1,to,"[-0.36955663561820984, -1.0432010889053345, 0...."
2,a,"[0.23711362481117249, -0.09444977343082428, -0..."
3,of,"[-1.24164879322052, 1.2665270566940308, -0.016..."
4,and,"[-0.7918386459350586, 0.2583085596561432, 0.27..."


# **Experiment 3:** Using BERT# 

In [23]:
pip install transformers torch pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 7.0 MB/s eta 0:00:000:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 20.2 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5

In [24]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [27]:
from transformers import BertTokenizer, BertModel

import torch


dataset = load_dataset("lucadiliello/newsqa")  

texts = dataset["train"]["context"][:20] 

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

word_embeddings = {}

for text in texts:
    
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)

    last_hidden = outputs.last_hidden_state.squeeze(0)
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'].squeeze(0))

    for token, emb in zip(tokens, last_hidden):
        word_embeddings[token] = emb.tolist()  


df = pd.DataFrame(list(word_embeddings.items()), columns=['token', 'embedding'])
df.to_csv('bert_embeddings_newsqa.csv', index=False)


In [28]:
df_bert = pd.read_csv('bert_embeddings_newsqa.csv')
df_bert.shape
df_bert.head()

,token,embedding
0,[CLS],"[-0.46617886424064636, -0.17095321416854858, -..."
1,new,"[-0.3519260883331299, -1.258732795715332, -0.6..."
2,delhi,"[-0.8275781273841858, -1.0234425067901611, -0...."
3,",","[-0.41135963797569275, -0.05573553591966629, 0..."
4,india,"[-0.4988032281398773, -0.5487556457519531, -0...."
